This project evaluates 


In [38]:
# check installed version
import pycaret
pycaret.__version__

'3.1.0'

In [39]:
import pandas as pd
from sklearn.model_selection import train_test_split
from pycaret.classification import *
from imblearn.over_sampling import *
from imblearn.under_sampling import *
from sklearn.metrics import confusion_matrix
import seaborn as sns
import numpy as np
import os

In [77]:
data = pd.read_csv("..\diabetes_cleaned.csv")

In [78]:
data.drop(['Unnamed: 0','heart_attack','angina_or_chd','chd_mi'], axis=1, inplace=True)

#data.drop(['income','age','sleep_time'], axis=1, inplace=True)

In [42]:
data.loc[data.diabetes.isin([2]), "diabetes"] = 1
data.diabetes.value_counts()

0.0    254681
1.0     48272
Name: diabetes, dtype: int64

In [80]:
6793/300000

0.022643333333333335

In [79]:
data.diabetes.value_counts()

0.0    254681
2.0     41479
1.0      6793
Name: diabetes, dtype: int64

In [43]:
TARGET = 'diabetes'
ANSWER = 42

In [44]:
data.diabetes.value_counts()

0.0    254681
1.0     48272
Name: diabetes, dtype: int64

In [45]:
#split data into 20% split with equal parts diabetes labels.
train_df, test_df = train_test_split(data, test_size=0.2, random_state=42, stratify=data['diabetes'])

In [46]:
# The unbalanced ratio
print("Train")
print(train_df.diabetes.value_counts())
print("Test")
test_df.diabetes.value_counts()

Train
0.0    203744
1.0     38618
Name: diabetes, dtype: int64
Test


0.0    50937
1.0     9654
Name: diabetes, dtype: int64

In [47]:
test_labels = test_df[TARGET]
test_features = test_df.drop(columns=TARGET)

In [48]:
# #binary
# binary_data = train_df.copy()
# binary_data.loc[binary_data.diabetes.isin([2]), TARGET] = 1
# binary_data.diabetes.value_counts()
# data = binary_data

In [49]:
# over_sampled_data = OverSample(train_df, TARGET, "RandomOverSampler")

# print(over_sampled_data.diabetes.value_counts())

In [50]:
# over_sampled_binary_data = OverSample(binary_data, TARGET, "RandomOverSampler")

# print(over_sampled_binary_data.diabetes.value_counts())

In [51]:
def Sampler(df, target, sampler_name, strategy):
    sampler = globals()[sampler_name](sampling_strategy=strategy)

    under_train_labels = df[target]
    under_train_features = df.drop(columns=target)
    
    X, y = sampler.fit_resample(under_train_features, under_train_labels)
    
    sampled_data = X
    sampled_data[target] = y
    
    print(sampled_data[target].value_counts())
    return sampled_data    

In [52]:
# def UnderSample(df, target, sampler_name):
#     sampler = globals()[sampler_name]
#     under_sampler = sampler()

#     under_train_labels = df[target]
#     under_train_features = df.drop(columns=target)
    
#     X_under, y_under = under_sampler.fit_resample(under_train_features, under_train_labels)
    
#     under_sampled_data = X_under
#     under_sampled_data[TARGET] = y_under
#     return under_sampled_data    

# def OverSample(df, target, sampler_name):
#     sampler = globals()[sampler_name]
#     over_sampler = sampler(random_state=ANSWER)
    
#     train_labels = df[target].copy()
#     train_features = df.drop(columns=target)

#     over_sampled_features, over_sampled_labels = over_sampler.fit_resample(train_features, train_labels)

#     over_sampled_data = over_sampled_features
#     over_sampled_data[target] = over_sampled_labels
    
#     return over_sampled_data

In [53]:
# under_sampled_data = UnderSample(train_df, TARGET, "RandomUnderSampler")
# print(under_sampled_data.diabetes.value_counts())

In [54]:
# under_sampled_binary_data = UnderSample(binary_data, TARGET, "RandomUnderSampler")

# print(under_sampled_binary_data.diabetes.value_counts())

In [55]:
# s = setup(under_sampled_data, target = TARGET, session_id = 123)
# best = compare_models()

In [56]:
#datasets = [under_sampled_data, over_sampled_data, under_sampled_binary_data, over_sampled_binary_data]
# datasets = [over_sampled_data]
# for idx, data in enumerate(datasets):

if False:
    over_samplers = ['RandomOverSampler','SMOTE','SMOTEN','ADASYN','BorderlineSMOTE','KMeansSMOTE','SVMSMOTE']
    for sampler in over_samplers:
        over_sampled_data = OverSample(train_df, TARGET, sampler)
        s = setup(over_sampled_data, target = TARGET, session_id = 123)
        best = compare_models()
        test_acc = best.score(test_features,  test_labels)

        print(f'\nSampler: {sampler} Test accuracy:', test_acc)


In [57]:
#  test_acc = best.score(test_features,  test_labels)
# print(f'\nSampler: {sampler}, Strategy: {strategy}, Test accuracy:', test_acc)

In [58]:
# sampled_data.diabetes.value_counts()

In [59]:
import matplotlib.pyplot as plt

def print_test_heat_map(model, sampler, strategy):
    predictions = predict_model(model, data=test_features)
    cm = confusion_matrix(test_labels, predictions["prediction_label"])
    cm_total = cm.sum(axis=1)
    result = [[0.0,0.0,0.0],[0.0,0.0,0.0]]    
    for row in range(0,2):
        for column in range(0,2):
            result[row][column] = cm[row][column]/(cm_total[row])
            
    total_percent = result[0][0] + result[1][1]
    diabetes_percent = result[1][1]
    # print(f"Total Percent Accurate {total_percent*100}")
    # print(f"Diabetes Percent Accurate {diabetes_percent*100}")
    
    print(f"{result[0][0]*100},{result[1][1]*100},{type(model).__name__},{sampler},{strategy},{total_percent*100},{diabetes_percent*100}")
    # sns.heatmap(result, annot=True, 
    #             fmt='.2%', cmap='Blues')
    # plt.show()
    # plt.figure()


In [60]:
print(f"{100:.2f}")

100.00


In [67]:
samplers_working = pd.read_csv(r"..\3label.csv")
samplers_working = samplers_working.drop_duplicates(subset=['Sampler','type'])[['Sampler','type']].sort_values(by='Sampler')

In [69]:
samplers_working.drop([70], axis =0, inplace=True)
samplers_working.drop([42], axis =0, inplace=True)
samplers_working.drop([28], axis =0, inplace=True)


In [70]:
samplers_working

,Sampler,type
98,ADASYN,auto
56,RandomOverSampler,auto
14,SMOTE,all
0,TomekLinks,all


In [ ]:
# Calculate without sampler
s = setup(train_df, target = TARGET, session_id = 123,verbose=False)
best = compare_models(n_select=14,verbose=False)
for model in best:    

    #print(f'\nSampler: {sampler}, Strategy: {strategy}')
    print_test_heat_map(model, "Base", "None")

In [74]:
import matplotlib.pyplot as plt

def print_test_heat_map3(model, sampler, strategy):
    predictions = predict_model(model, data=test_features)
    cm = confusion_matrix(test_labels, predictions["prediction_label"])
    cm_total = cm.sum(axis=1)
    result = [[0.0,0.0,0.0],[0.0,0.0,0.0],[0.0,0.0,0.0]]    
    for row in range(0,3):
        for column in range(0,3):
            result[row][column] = cm[row][column]/(cm_total[row])
            
    total_percent = result[0][0] + result[1][1] + result[2][2]
    diabetes_percent = result[1][1] + result[2][2]
    # print(f"Total Percent Accurate {total_percent*100}")
    # print(f"Diabetes Percent Accurate {diabetes_percent*100}")
    
    print(f"{result[0][0]*100},{result[1][1]*100},{result[2][2]*100},{type(model).__name__},{sampler},{strategy},{total_percent*100},{diabetes_percent*100}")
    # sns.heatmap(result, annot=True, 
    #             fmt='.2%', cmap='Blues')
    # plt.show()
    # plt.figure()

In [71]:
s = setup(sampled_data, target = TARGET, session_id = 123,verbose=False)
best = compare_models(n_select=14)        

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.8606,0.8178,0.1379,0.5599,0.2213,0.1744,0.2273,0.2600
gbc,Gradient Boosting Classifier,0.8604,0.8171,0.1420,0.5553,0.2262,0.1781,0.2292,1.8780
ada,Ada Boost Classifier,0.8597,0.8146,0.1532,0.5410,0.2388,0.1872,0.2333,0.6280
lr,Logistic Regression,0.8580,0.8094,0.1339,0.5229,0.2131,0.1642,0.2112,1.1560
ridge,Ridge Classifier,0.8570,0.0000,0.0172,0.5767,0.0333,0.0252,0.0808,0.0470
lda,Linear Discriminant Analysis,0.8565,0.8054,0.1629,0.5021,0.2460,0.1889,0.2259,0.1230
qda,Quadratic Discriminant Analysis,0.8563,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0990
dummy,Dummy Classifier,0.8563,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0400
svm,SVM - Linear Kernel,0.8551,0.0000,0.0023,0.0373,0.0043,0.0008,0.0021,0.2750
rf,Random Forest Classifier,0.8494,0.7736,0.1844,0.4422,0.2602,0.1919,0.2148,1.5330


TypeError: setup() got an unexpected keyword argument 'class_weight'

In [65]:
#Print working types.
for index, row in samplers_working.iterrows():
    sampled_data = Sampler(train_df, TARGET, row.Sampler, row.type)
    sampled_data.diabetes.value_counts()
    s = setup(sampled_data, target = TARGET, session_id = 123,verbose=False)
    best = compare_models(n_select=14,verbose=False)
    for model in best:    

        #print(f'\nSampler: {sampler}, Strategy: {strategy}')
        print_test_heat_map(model, row.Sampler, row.type)

0.0    203744
1.0    200891
Name: diabetes, dtype: int64


92.3061821465732,25.605966438781852,ExtraTreesClassifier,ADASYN,auto,117.91214858535506,25.605966438781852


92.76557315900034,24.487259167184586,RandomForestClassifier,ADASYN,auto,117.25283232618493,24.487259167184586


95.32756149753617,23.45141910089082,LGBMClassifier,ADASYN,auto,118.778980598427,23.45141910089082


85.90611932387067,31.95566604516263,DecisionTreeClassifier,ADASYN,auto,117.86178536903329,31.95566604516263


...

95.6711231521291,17.184586699813547,RandomForestClassifier,TomekLinks,all,112.85570985194265,17.184586699813547


95.1096452480515,18.489745183343693,ExtraTreesClassifier,TomekLinks,all,113.5993904313952,18.489745183343693


94.7091505192689,19.81562046819971,KNeighborsClassifier,TomekLinks,all,114.5247709874686,19.81562046819971


88.19522154818698,29.11746426351771,DecisionTreeClassifier,TomekLinks,all,117.31268581170468,29.11746426351771


84.09996662543927,48.746633519784545,GaussianNB,TomekLinks,all,132.84660014522382,48.746633519784545


None,Diab,Classifier,Sampler,Mode,Total,Diab2
92.3061821465732,25.605966438781852,ExtraTreesClassifier,ADASYN,auto,117.91214858535506,25.605966438781852
92.76557315900034,24.487259167184586,RandomForestClassifier,ADASYN,auto,117.25283232618493,24.487259167184586
95.32756149753617,23.45141910089082,LGBMClassifier,ADASYN,auto,118.778980598427,23.45141910089082
85.90611932387067,31.95566604516263,DecisionTreeClassifier,ADASYN,auto,117.86178536903329,31.95566604516263
84.61825392151088,53.42863061943236,GradientBoostingClassifier,ADASYN,auto,138.04688454094324,53.42863061943236
68.00950193376131,63.807748083695884,KNeighborsClassifier,ADASYN,auto,131.8172500174572,63.807748083695884
77.74113120128786,64.51211932877564,AdaBoostClassifier,ADASYN,auto,142.2532505300635,64.51211932877564
67.93489997447828,78.56846902838201,RidgeClassifier,ADASYN,auto,146.5033690028603,78.56846902838201
67.93489997447828,78.56846902838201,LinearDiscriminantAnalysis,ADASYN,auto,146.5033690028603,78.56846902838201
69.00877554626304,77.46012015744769,LogisticRegression,ADASYN,auto,146.4688957037107,77.46012015744769
75.457918605336,68.93515641185,SGDClassifier,ADASYN,auto,144.39307501718602,68.93515641185
65.53782123014705,77.8744561839652,GaussianNB,ADASYN,auto,143.41227741411222,77.8744561839652
100.0,0.0,QuadraticDiscriminantAnalysis,ADASYN,auto,100.0,0.0
100.0,0.0,DummyClassifier,ADASYN,auto,100.0,0.0
88.37976323693975,32.17319245908431,ExtraTreesClassifier,RandomOverSampler,auto,120.55295569602407,32.17319245908431
84.15297328071932,44.98653407913818,RandomForestClassifier,RandomOverSampler,auto,129.1395073598575,44.98653407913818
80.83318609262422,39.030453697949035,DecisionTreeClassifier,RandomOverSampler,auto,119.86363979057326,39.030453697949035
68.9695113571667,64.78143774601202,KNeighborsClassifier,RandomOverSampler,auto,133.7509491031787,64.78143774601202
69.3346683157626,78.8688626476072,LGBMClassifier,RandomOverSampler,auto,148.20353096336981,78.8688626476072
69.83528672674088,78.3716594157862,GradientBoostingClassifier,RandomOverSampler,auto,148.20694614252707,78.3716594157862
71.26646641930228,76.29997928319867,AdaBoostClassifier,RandomOverSampler,auto,147.56644570250094,76.29997928319867
70.25737675952647,76.52786409778331,RidgeClassifier,RandomOverSampler,auto,146.78524085730976,76.52786409778331
70.25737675952647,76.52786409778331,LinearDiscriminantAnalysis,RandomOverSampler,auto,146.78524085730976,76.52786409778331
71.09174077782359,75.75098404806297,LogisticRegression,RandomOverSampler,auto,146.84272482588656,75.75098404806297
75.45202897697155,69.68096125958152,SGDClassifier,RandomOverSampler,auto,145.13299023655307,69.68096125958152
74.72171505977973,66.03480422622748,GaussianNB,RandomOverSampler,auto,140.75651928600718,66.03480422622748
100.0,0.0,QuadraticDiscriminantAnalysis,RandomOverSampler,auto,100.0,0.0
0.0,100.0,DummyClassifier,RandomOverSampler,auto,100.0,100.0
92.19035278873903,26.217112077895173,ExtraTreesClassifier,SMOTE,all,118.4074648666342,26.217112077895173
92.7145297131751,25.38844002486016,RandomForestClassifier,SMOTE,all,118.10296973803527,25.38844002486016
94.50497673596796,26.413921690490987,LGBMClassifier,SMOTE,all,120.91889842645895,26.413921690490987
85.77458429039794,31.997099647814377,DecisionTreeClassifier,SMOTE,all,117.7716839382123,31.997099647814377
84.10192983489408,55.12740832815414,GradientBoostingClassifier,SMOTE,all,139.22933816304823,55.12740832815414
70.24167108388794,62.48187279883985,KNeighborsClassifier,SMOTE,all,132.7235438827278,62.48187279883985
78.63046508431985,63.90097368966232,AdaBoostClassifier,SMOTE,all,142.5314387739822,63.90097368966232
69.71749415945187,76.9939921276155,RidgeClassifier,SMOTE,all,146.71148628706737,76.9939921276155
69.71749415945187,76.9939921276155,LinearDiscriminantAnalysis,SMOTE,all,146.71148628706737,76.9939921276155
71.03677091308872,75.69919204474829,LogisticRegression,SMOTE,all,146.735962957837,75.69919204474829
73.65176590690461,71.45224777294385,SGDClassifier,SMOTE,all,145.10401367984846,71.45224777294385
68.24508706833932,75.07768800497203,GaussianNB,SMOTE,all,143.32277507331133,75.07768800497203
100.0,0.0,QuadraticDiscriminantAnalysis,SMOTE,all,100.0,0.0
0.0,100.0,DummyClassifier,SMOTE,all,100.0,100.0
97.93666686298761,13.963124093639943,LGBMClassifier,TomekLinks,all,111.89979095662754,13.963124093639943
97.81691108624379,14.605344934742076,GradientBoostingClassifier,TomekLinks,all,112.42225602098588,14.605344934742076
97.57347311384652,15.092189765900146,AdaBoostClassifier,TomekLinks,all,112.66566287974666,15.092189765900146
97.71482419459332,13.414128858504245,LogisticRegression,TomekLinks,all,111.12895305309756,13.414128858504245
99.79189979778943,1.823078516677025,RidgeClassifier,TomekLinks,all,101.61497831446647,1.823078516677025
97.01788483813338,15.910503418272217,LinearDiscriminantAnalysis,TomekLinks,all,112.92838825640561,15.910503418272217
100.0,0.0,QuadraticDiscriminantAnalysis,TomekLinks,all,100.0,0.0
100.0,0.0,DummyClassifier,TomekLinks,all,100.0,0.0
100.0,0.0,SGDClassifier,TomekLinks,all,100.0,0.0
95.6711231521291,17.184586699813547,RandomForestClassifier,TomekLinks,all,112.85570985194265,17.184586699813547
95.1096452480515,18.489745183343693,ExtraTreesClassifier,TomekLinks,all,113.5993904313952,18.489745183343693
94.7091505192689,19.81562046819971,KNeighborsClassifier,TomekLinks,all,114.5247709874686,19.81562046819971
88.19522154818698,29.11746426351771,DecisionTreeClassifier,TomekLinks,all,117.31268581170468,29.11746426351771
84.09996662543927,48.746633519784545,GaussianNB,TomekLinks,all,132.84660014522382,48.746633519784545
97.75212517423483,14.605344934742076,GradientBoostingClassifier,Base,None,112.35747010897693,14.605344934742076
97.98574709935804,13.714522477729437,LGBMClassifier,Base,None,111.70026957708747,13.714522477729437
97.33199835090406,16.4284234514191,AdaBoostClassifier,Base,None,113.76042180232315,16.4284234514191
99.57987317666922,3.1489538015330436,RidgeClassifier,Base,None,102.72882697820226,3.1489538015330436
97.54402497202427,14.23244251087632,LogisticRegression,Base,None,111.77646748290057,14.23244251087632
96.86279129120285,16.65630826600373,LinearDiscriminantAnalysis,Base,None,113.51909955720659,16.65630826600373
100.0,0.0,QuadraticDiscriminantAnalysis,Base,None,100.0,0.0
100.0,0.0,DummyClassifier,Base,None,100.0,0.0
100.0,0.0,SGDClassifier,Base,None,100.0,0.0
95.36878889608732,17.83716594157862,RandomForestClassifier,Base,None,113.20595483766593,17.83716594157862
94.72485619490743,18.645121193287757,ExtraTreesClassifier,Base,None,113.36997738819518,18.645121193287757
94.23798025011288,19.618810855603893,KNeighborsClassifier,Base,None,113.85679110571678,19.618810855603893
84.14904686180968,47.866169463434844,GaussianNB,Base,None,132.01521632524452,47.866169463434844
86.83471739599898,30.909467578205923,DecisionTreeClassifier,Base,None,117.74418497420491,30.909467578205923

None,Pre,Diab,Classifier,Sampler,Mode,Total,Diab2
92.4689806816397,1.030169242089772,25.241080038572804,ExtraTreesClassifier,ADASYN,auto,118.7402299623023,26.27124928066258
93.00102088895869,0.5886681383370125,24.65043394406943,RandomForestClassifier,ADASYN,auto,118.24012297136515,25.239102082406443
85.67614261033454,3.8263428991905815,29.050144648023146,DecisionTreeClassifier,ADASYN,auto,118.55263015754826,32.87648754721373
62.91031883147479,17.51287711552612,52.09739633558341,KNeighborsClassifier,ADASYN,auto,132.52059228258432,69.61027345110952
94.64229621485785,0.0,28.2907425265188,LGBMClassifier,ADASYN,auto,122.93303874137666,28.2907425265188
82.94919114182504,0.22075055187637968,59.72757955641272,GradientBoostingClassifier,ADASYN,auto,142.89752125011415,59.948330108289106
74.79974870425632,5.518763796909492,66.04387656702025,AdaBoostClassifier,ADASYN,auto,146.36238906818605,71.56264036392975
63.642610334537466,32.00883002207506,58.20877531340405,LogisticRegression,ADASYN,auto,153.86021567001657,90.2176053354791
62.11716664049003,36.12950699043414,55.81002892960463,LinearDiscriminantAnalysis,ADASYN,auto,154.05670256052878,91.93953592003876
66.86233705041622,27.740986019131714,58.57039537126326,RidgeClassifier,ADASYN,auto,153.1737184408112,86.31138139039498
58.44392963719177,26.122148638704928,66.18852459016394,GaussianNB,ADASYN,auto,150.75460286606065,92.31067322886885
66.73276268258206,19.793966151582048,64.99517839922855,SGDClassifier,ADASYN,auto,151.52190723339265,84.78914455081059
100.0,0.0,0.0,QuadraticDiscriminantAnalysis,ADASYN,auto,100.0,0.0
100.0,0.0,0.0,DummyClassifier,ADASYN,auto,100.0,0.0
11.174807601696246,76.52685798381162,20.16634522661524,LGBMClassifier,NearMiss,all,107.86801081212312,96.69320321042686
7.788204806031097,71.22884473877852,24.361137897782065,ExtraTreesClassifier,NearMiss,all,103.37818744259168,95.58998263656058
8.683445892885189,71.22884473877852,24.493731918997106,RandomForestClassifier,NearMiss,all,104.40602255066081,95.72257665777563
17.154861002041777,74.02501839587933,20.118129218900673,GradientBoostingClassifier,NearMiss,all,111.29800861682178,94.14314761477999
15.963169467567143,73.65710080941868,21.335583413693346,AdaBoostClassifier,NearMiss,all,110.95585369067918,94.99268422311204
20.137034710224597,71.00809418690214,21.96239151398264,LogisticRegression,NearMiss,all,113.10752041110939,92.97048570088478
12.425396576095492,65.04782928623987,26.97685631629701,DecisionTreeClassifier,NearMiss,all,104.45008217863239,92.02468560253689
29.566514842154863,62.76674025018396,26.458534233365476,LinearDiscriminantAnalysis,NearMiss,all,118.79178932570431,89.22527448354944
30.734647400659654,67.91758646063282,20.527965284474444,RidgeClassifier,NearMiss,all,119.18019914576692,88.44555174510727
27.613083084655255,54.81972038263429,31.0390549662488,KNeighborsClassifier,NearMiss,all,113.47185843353833,85.85877534888309
20.03298256635778,82.33995584988962,6.159594985535198,SGDClassifier,NearMiss,all,108.53253340178262,88.49955083542483
32.99827234176222,46.799116997792495,34.01639344262295,GaussianNB,NearMiss,all,113.81378278217767,80.81551044041544
100.0,0.0,0.0,QuadraticDiscriminantAnalysis,NearMiss,all,100.0,0.0
100.0,0.0,0.0,DummyClassifier,NearMiss,all,100.0,0.0
11.174807601696246,76.52685798381162,20.16634522661524,LGBMClassifier,NearMiss,all,107.86801081212312,96.69320321042686
7.788204806031097,71.22884473877852,24.361137897782065,ExtraTreesClassifier,NearMiss,all,103.37818744259168,95.58998263656058
8.683445892885189,71.22884473877852,24.493731918997106,RandomForestClassifier,NearMiss,all,104.40602255066081,95.72257665777563
17.154861002041777,74.02501839587933,20.118129218900673,GradientBoostingClassifier,NearMiss,all,111.29800861682178,94.14314761477999
15.963169467567143,73.65710080941868,21.335583413693346,AdaBoostClassifier,NearMiss,all,110.95585369067918,94.99268422311204
20.137034710224597,71.00809418690214,21.96239151398264,LogisticRegression,NearMiss,all,113.10752041110939,92.97048570088478
12.425396576095492,65.04782928623987,26.97685631629701,DecisionTreeClassifier,NearMiss,all,104.45008217863239,92.02468560253689
29.566514842154863,62.76674025018396,26.458534233365476,LinearDiscriminantAnalysis,NearMiss,all,118.79178932570431,89.22527448354944
30.734647400659654,67.91758646063282,20.527965284474444,RidgeClassifier,NearMiss,all,119.18019914576692,88.44555174510727
27.613083084655255,54.81972038263429,31.0390549662488,KNeighborsClassifier,NearMiss,all,113.47185843353833,85.85877534888309
20.03298256635778,82.33995584988962,6.159594985535198,SGDClassifier,NearMiss,all,108.53253340178262,88.49955083542483
32.99827234176222,46.799116997792495,34.01639344262295,GaussianNB,NearMiss,all,113.81378278217767,80.81551044041544
100.0,0.0,0.0,QuadraticDiscriminantAnalysis,NearMiss,all,100.0,0.0
100.0,0.0,0.0,DummyClassifier,NearMiss,all,100.0,0.0
87.90246583948485,2.57542310522443,30.364030858244938,ExtraTreesClassifier,RandomOverSampler,all,120.84191980295422,32.93945396346937
83.90136642060625,2.57542310522443,43.7801350048216,RandomForestClassifier,RandomOverSampler,all,130.25692453065227,46.35555811004603
78.84403957907963,7.358351729212656,35.450819672131146,DecisionTreeClassifier,RandomOverSampler,all,121.65321098042344,42.809171401343804
63.725066750431914,16.703458425312732,54.24300867888139,KNeighborsClassifier,RandomOverSampler,all,134.67153385462603,70.94646710419413
62.32527092822365,28.550404709345106,65.92333654773384,LGBMClassifier,RandomOverSampler,all,156.7990121853026,94.47374125707894
62.284042720276425,32.450331125827816,63.09064609450338,GradientBoostingClassifier,RandomOverSampler,all,157.82501994060763,95.54097722033119
64.20213601382126,31.714495952906546,60.342333654773384,AdaBoostClassifier,RandomOverSampler,all,156.2589656215012,92.05682960767993
64.4691377414795,31.42016188373804,59.36595949855352,LogisticRegression,RandomOverSampler,all,155.25525912377108,90.78612138229157
62.88479660750747,35.246504782928625,56.231918997107044,LinearDiscriminantAnalysis,RandomOverSampler,all,154.36322038754312,91.47842378003567
68.26998586461441,22.958057395143488,61.36692381870781,RidgeClassifier,RandomOverSampler,all,152.59496707846571,84.3249812138513
67.10185330610963,9.271523178807946,75.20491803278688,SGDClassifier,RandomOverSampler,all,151.57829451770445,84.47644121159483
69.37529448719962,17.58646063281825,60.92092574734812,GaussianNB,RandomOverSampler,all,147.882680867366,78.50738638016637
100.0,0.0,0.0,QuadraticDiscriminantAnalysis,RandomOverSampler,all,100.0,0.0
100.0,0.0,0.0,DummyClassifier,RandomOverSampler,all,100.0,0.0
87.90246583948485,2.57542310522443,30.364030858244938,ExtraTreesClassifier,RandomOverSampler,auto,120.84191980295422,32.93945396346937
83.90136642060625,2.57542310522443,43.7801350048216,RandomForestClassifier,RandomOverSampler,auto,130.25692453065227,46.35555811004603
78.84403957907963,7.358351729212656,35.450819672131146,DecisionTreeClassifier,RandomOverSampler,auto,121.65321098042344,42.809171401343804
63.725066750431914,16.703458425312732,54.24300867888139,KNeighborsClassifier,RandomOverSampler,auto,134.67153385462603,70.94646710419413
62.32527092822365,28.550404709345106,65.92333654773384,LGBMClassifier,RandomOverSampler,auto,156.7990121853026,94.47374125707894
62.284042720276425,32.450331125827816,63.09064609450338,GradientBoostingClassifier,RandomOverSampler,auto,157.82501994060763,95.54097722033119
64.20213601382126,31.714495952906546,60.342333654773384,AdaBoostClassifier,RandomOverSampler,auto,156.2589656215012,92.05682960767993
64.4691377414795,31.42016188373804,59.36595949855352,LogisticRegression,RandomOverSampler,auto,155.25525912377108,90.78612138229157
62.88479660750747,35.246504782928625,56.231918997107044,LinearDiscriminantAnalysis,RandomOverSampler,auto,154.36322038754312,91.47842378003567
68.26998586461441,22.958057395143488,61.36692381870781,RidgeClassifier,RandomOverSampler,auto,152.59496707846571,84.3249812138513
67.10185330610963,9.271523178807946,75.20491803278688,SGDClassifier,RandomOverSampler,auto,151.57829451770445,84.47644121159483
69.37529448719962,17.58646063281825,60.92092574734812,GaussianNB,RandomOverSampler,auto,147.882680867366,78.50738638016637
100.0,0.0,0.0,QuadraticDiscriminantAnalysis,RandomOverSampler,auto,100.0,0.0
100.0,0.0,0.0,DummyClassifier,RandomOverSampler,auto,100.0,0.0
92.42578922569498,0.9565857247976454,26.12102217936355,ExtraTreesClassifier,SMOTE,all,119.50339712985618,27.077607904161194
92.98924140097377,0.5886681383370125,25.891996142719382,RandomForestClassifier,SMOTE,all,119.46990568203016,26.48066428105639
85.43466310664363,4.415011037527594,29.6648987463838,DecisionTreeClassifier,SMOTE,all,119.51457289055503,34.0799097839114
64.23551122977854,17.880794701986755,51.88042430086789,KNeighborsClassifier,SMOTE,all,133.99673023263318,69.76121900285465
93.54091408826764,0.0,32.73866923818708,LGBMClassifier,SMOTE,all,126.27958332645473,32.73866923818708
83.08858174964662,0.44150110375275936,59.763741562198646,GradientBoostingClassifier,SMOTE,all,143.29382441559804,60.20524266595141
76.77477618972829,3.3112582781456954,66.32111861137898,AdaBoostClassifier,SMOTE,all,146.40715307925296,69.63237688952468
63.8330453902937,33.99558498896248,57.58196721311475,LogisticRegression,SMOTE,all,155.41059759237092,91.57755220207721
62.14268886445736,37.38042678440029,54.68900675024108,LinearDiscriminantAnalysis,SMOTE,all,154.21212239909875,92.06943353464136
67.3649285377729,26.490066225165563,59.18514946962391,RidgeClassifier,SMOTE,all,153.04014423256237,85.67521569478947
58.481231349144025,30.31640912435614,61.620057859209254,GaussianNB,SMOTE,all,150.41769833270942,91.9364669835654
75.378906863515,17.071376011773364,56.84667309546769,SGDClassifier,SMOTE,all,149.29695597075604,73.91804910724106
100.0,0.0,0.0,QuadraticDiscriminantAnalysis,SMOTE,all,100.0,0.0
100.0,0.0,0.0,DummyClassifier,SMOTE,all,100.0,0.0
98.5452332338621,0.0,11.30665380906461,LGBMClassifier,TomekLinks,all,109.85188704292672,11.30665380906461
98.40780587403802,0.0,12.704918032786885,GradientBoostingClassifier,TomekLinks,all,111.1127239068249,12.704918032786885
98.00337678655568,0.0,14.103182256509161,AdaBoostClassifier,TomekLinks,all,112.10655904306483,14.103182256509161
98.14080414637976,0.0,12.270973963355834,LogisticRegression,TomekLinks,all,110.4117781097356,12.270973963355834
99.90380084812314,0.0,0.904050144648023,RidgeClassifier,TomekLinks,all,100.80785099277117,0.904050144648023
100.0,0.0,0.0,QuadraticDiscriminantAnalysis,TomekLinks,all,100.0,0.0
100.0,0.0,0.0,DummyClassifier,TomekLinks,all,100.0,0.0
100.0,0.0,0.0,SGDClassifier,TomekLinks,all,100.0,0.0
97.40262289932464,0.0,16.224686595949855,LinearDiscriminantAnalysis,TomekLinks,all,113.62730949527449,16.224686595949855
96.39940317260877,0.22075055187637968,15.17598842815815,RandomForestClassifier,TomekLinks,all,111.79614215264331,15.396738980034531
96.13436469294801,0.14716703458425312,15.838958534233367,KNeighborsClassifier,TomekLinks,all,112.12049026176562,15.986125568817618
95.76134757342547,0.36791758646063283,15.814850530376084,ExtraTreesClassifier,TomekLinks,all,111.9441156902622,16.18276811683672
88.02615046332653,3.237674760853569,26.711668273866923,DecisionTreeClassifier,TomekLinks,all,117.97549349804702,29.949343034720492
84.24886131616147,1.177336276674025,50.01205400192864,GaussianNB,TomekLinks,all,135.43825159476413,51.189390278602666
98.39995288204805,0.0,12.258919961427193,LGBMClassifier,Base,None,110.65887284347524,12.258919961427193
98.08190670645516,0.0,14.163452266152362,GradientBoostingClassifier,Base,None,112.24535897260752,14.163452266152362
97.80116224281451,0.0,15.537608486017357,AdaBoostClassifier,Base,None,113.33877072883188,15.537608486017357
97.88950840270144,0.0,13.729508196721312,LogisticRegression,Base,None,111.61901659942275,13.729508196721312
99.79582220826134,0.0,1.95274831243973,RidgeClassifier,Base,None,101.74857052070108,1.95274831243973
100.0,0.0,0.0,QuadraticDiscriminantAnalysis,Base,None,100.0,0.0
100.0,0.0,0.0,DummyClassifier,Base,None,100.0,0.0
100.0,0.0,0.0,SGDClassifier,Base,None,100.0,0.0
97.25930579550808,0.0,17.044358727097396,LinearDiscriminantAnalysis,Base,None,114.30366452260547,17.044358727097396
95.9400031411968,0.14716703458425312,16.36933461909354,RandomForestClassifier,Base,None,112.45650479487459,16.516501653677793
95.61410397361395,0.36791758646063283,17.369816779170684,KNeighborsClassifier,Base,None,113.35183833924528,17.73773436563132
95.16255693419193,0.14716703458425312,17.514464802314368,ExtraTreesClassifier,Base,None,112.82418877109055,17.66163183689862
84.00149206847809,2.7225901398086827,48.98746383799421,GaussianNB,Base,None,135.711546046281,51.71005397780289
86.86783414480918,3.7527593818984544,27.603664416586305,DecisionTreeClassifier,Base,None,118.22425794329394,31.356423798484755

In [ ]:
samplers = ['SMOTE','ADASYN']
# samplers = ['RandomOverSampler', 'EditedNearestNeighbours','RepeatedEditedNearestNeighbours','AllKNN','NeighbourhoodCleaningRule','OneSidedSelection','TomekLinks', 'SMOTE','ADASYN']

# under_samplers = ['RepeatedEditedNearestNeighbours',]
# over_samplers = ['RandomOverSampler', 'SMOTE','ADASYN']

# working_sampler = ['SMOTE']

#samplers = ['TomekLinks','SMOTE','RandomOverSampler','NearMiss']

for sampler in samplers:
    strategies = ['auto']
    for strategy in strategies:
        sampled_data = Sampler(train_df, TARGET, sampler, strategy)
        sampled_data.diabetes.value_counts()
        s = setup(sampled_data, target = TARGET, session_id = 123,verbose=False)
        best = compare_models(n_select=14,verbose=False)
        for model in best:    

            #print(f'\nSampler: {sampler}, Strategy: {strategy}')
            print_test_heat_map(model, sampler, strategy)
            
        # test_acc = best.score(test_features,  test_labels)


        # #evaluate_model(best)
        # plot_model(best, plot = 'auc')
        # plot_model(best, plot = 'confusion_matrix', plot_kwargs = {'percent' : True})
        # print_test_heat_map()
        # plot_model(best, plot = 'feature')


TomekLinks,LGBMClassifier,all,109.852,11.307
TomekLinks,GradientBoostingClassifier,all,111.113,12.705
TomekLinks,AdaBoostClassifier,all,112.107,14.103
TomekLinks,LogisticRegression,all,110.422,12.283
TomekLinks,RidgeClassifier,all,100.808,0.904
TomekLinks,QuadraticDiscriminantAnalysis,all,100.000,0.000
TomekLinks,DummyClassifier,all,100.000,0.000
TomekLinks,SGDClassifier,all,100.000,0.000
TomekLinks,LinearDiscriminantAnalysis,all,113.627,16.225
TomekLinks,RandomForestClassifier,all,111.796,15.397
TomekLinks,KNeighborsClassifier,all,112.120,15.986
TomekLinks,ExtraTreesClassifier,all,111.944,16.183
TomekLinks,DecisionTreeClassifier,all,117.975,29.949
TomekLinks,GaussianNB,all,135.438,51.189
SMOTE,ExtraTreesClassifier,all,119.503,27.078
SMOTE,RandomForestClassifier,all,119.470,26.481
SMOTE,DecisionTreeClassifier,all,119.515,34.080
SMOTE,KNeighborsClassifier,all,133.997,69.761
SMOTE,LGBMClassifier,all,126.280,32.739
SMOTE,GradientBoostingClassifier,all,143.294,60.205
SMOTE,AdaBoostClassifier,all,146.407,69.632
SMOTE,LogisticRegression,all,155.449,91.614
SMOTE,LinearDiscriminantAnalysis,all,154.212,92.069
SMOTE,RidgeClassifier,all,153.040,85.675
SMOTE,GaussianNB,all,150.418,91.936
SMOTE,SGDClassifier,all,149.297,73.918
SMOTE,QuadraticDiscriminantAnalysis,all,100.000,0.000
SMOTE,DummyClassifier,all,100.000,0.000
RandomOverSampler,ExtraTreesClassifier,all,120.842,32.939
RandomOverSampler,RandomForestClassifier,all,130.257,46.356
RandomOverSampler,DecisionTreeClassifier,all,121.653,42.809
RandomOverSampler,KNeighborsClassifier,all,134.672,70.946
RandomOverSampler,LGBMClassifier,all,156.799,94.474
RandomOverSampler,GradientBoostingClassifier,all,157.825,95.541
RandomOverSampler,AdaBoostClassifier,all,156.259,92.057
RandomOverSampler,LogisticRegression,all,155.351,90.884
RandomOverSampler,LinearDiscriminantAnalysis,all,154.363,91.478
RandomOverSampler,RidgeClassifier,all,152.595,84.325
RandomOverSampler,SGDClassifier,all,151.578,84.476
RandomOverSampler,GaussianNB,all,147.883,78.507
RandomOverSampler,QuadraticDiscriminantAnalysis,all,100.000,0.000
RandomOverSampler,DummyClassifier,all,100.000,0.000
NearMiss,LGBMClassifier,all,107.868,96.693
NearMiss,ExtraTreesClassifier,all,103.376,95.590
NearMiss,RandomForestClassifier,all,104.406,95.723
NearMiss,GradientBoostingClassifier,all,111.298,94.143
NearMiss,AdaBoostClassifier,all,110.956,94.993
NearMiss,LogisticRegression,all,113.049,92.922
NearMiss,DecisionTreeClassifier,all,104.450,92.025
NearMiss,LinearDiscriminantAnalysis,all,118.792,89.225
NearMiss,RidgeClassifier,all,119.180,88.446
NearMiss,KNeighborsClassifier,all,113.472,85.859
NearMiss,SGDClassifier,all,108.533,88.500
NearMiss,GaussianNB,all,113.814,80.816
NearMiss,QuadraticDiscriminantAnalysis,all,100.000,0.000
NearMiss,DummyClassifier,all,100.000,0.000
RandomOverSampler,ExtraTreesClassifier,auto,120.842,32.939
RandomOverSampler,RandomForestClassifier,auto,130.257,46.356
RandomOverSampler,DecisionTreeClassifier,auto,121.653,42.809
RandomOverSampler,KNeighborsClassifier,auto,134.672,70.946
RandomOverSampler,LGBMClassifier,auto,156.799,94.474
RandomOverSampler,GradientBoostingClassifier,auto,157.825,95.541
RandomOverSampler,AdaBoostClassifier,auto,156.259,92.057
RandomOverSampler,LogisticRegression,auto,155.351,90.884
RandomOverSampler,LinearDiscriminantAnalysis,auto,154.363,91.478
RandomOverSampler,RidgeClassifier,auto,152.595,84.325
RandomOverSampler,SGDClassifier,auto,151.578,84.476
RandomOverSampler,GaussianNB,auto,147.883,78.507
RandomOverSampler,QuadraticDiscriminantAnalysis,auto,100.000,0.000
RandomOverSampler,DummyClassifier,auto,100.000,0.000
RandomForestClassifier,AllKNN,auto,102.73105732810244,3.830476206695178
ExtraTreesClassifier,AllKNN,auto,102.42930781476834,3.7702061970519765
KNeighborsClassifier,AllKNN,auto,102.79987575252191,3.642109143443865
GradientBoostingClassifier,AllKNN,auto,102.77590582407268,3.0978784956605594
LGBMClassifier,AllKNN,auto,103.79888540311583,4.256322186530313
AdaBoostClassifier,AllKNN,auto,102.95808842272088,3.3389585342333654
RidgeClassifier,AllKNN,auto,100.0,0.0
QuadraticDiscriminantAnalysis,AllKNN,auto,100.0,0.0
DummyClassifier,AllKNN,auto,100.0,0.0
LogisticRegression,AllKNN,auto,101.75010454036509,1.9660618200886548
SGDClassifier,AllKNN,auto,100.09474328137536,0.10848601735776277
LinearDiscriminantAnalysis,AllKNN,auto,111.2661147257761,13.319672131147541
DecisionTreeClassifier,AllKNN,auto,110.31092974161871,17.19604046880577
GaussianNB,AllKNN,auto,132.41211590536,46.65548012712848
RandomForestClassifier,AllKNN,auto,102.73105732810244,3.830476206695178
ExtraTreesClassifier,AllKNN,auto,102.42930781476834,3.7702061970519765
KNeighborsClassifier,AllKNN,auto,102.79987575252191,3.642109143443865
GradientBoostingClassifier,AllKNN,auto,102.77590582407268,3.0978784956605594
LGBMClassifier,AllKNN,auto,103.79888540311583,4.256322186530313
AdaBoostClassifier,AllKNN,auto,102.95808842272088,3.3389585342333654
RidgeClassifier,AllKNN,auto,100.0,0.0
QuadraticDiscriminantAnalysis,AllKNN,auto,100.0,0.0
DummyClassifier,AllKNN,auto,100.0,0.0
LogisticRegression,AllKNN,auto,101.75010454036509,1.9660618200886548
SGDClassifier,AllKNN,auto,100.09474328137536,0.10848601735776277
LinearDiscriminantAnalysis,AllKNN,auto,111.2661147257761,13.319672131147541
DecisionTreeClassifier,AllKNN,auto,110.31092974161871,17.19604046880577
GaussianNB,AllKNN,auto,132.41211590536,46.65548012712848
ExtraTreesClassifier,SMOTE,auto,119.35053829999592,27.08966190608983
RandomForestClassifier,SMOTE,auto,119.37430394398785,26.398805278996484
DecisionTreeClassifier,SMOTE,auto,119.58443974774471,34.00056979329205
KNeighborsClassifier,SMOTE,auto,133.85153991715774,69.57087398343697
LGBMClassifier,SMOTE,auto,125.91517647876687,32.34088717454195
GradientBoostingClassifier,SMOTE,auto,143.1386250630391,59.56512105801318
AdaBoostClassifier,SMOTE,auto,147.91058443553914,73.11868872327275
LogisticRegression,SMOTE,auto,154.92010532501956,90.9457060789068
LinearDiscriminantAnalysis,SMOTE,auto,153.7499042307075,91.53653843834063
RidgeClassifier,SMOTE,auto,153.31489420770313,85.86358275804078
GaussianNB,SMOTE,auto,150.16552075782676,91.4899278569315
SGDClassifier,SMOTE,auto,152.92505514173868,94.59303064040367
QuadraticDiscriminantAnalysis,SMOTE,auto,100.0,0.0
DummyClassifier,SMOTE,auto,100.0,0.0
ExtraTreesClassifier,ADASYN,auto,118.23928912629312,25.78405118063583
RandomForestClassifier,ADASYN,auto,118.3693811748641,25.311426093978284
DecisionTreeClassifier,ADASYN,auto,119.26898880194202,33.63211115155721
KNeighborsClassifier,ADASYN,auto,132.68336694531905,69.70040793793724
LGBMClassifier,ADASYN,auto,123.20348419912885,28.700578592092572
GradientBoostingClassifier,ADASYN,auto,143.2988719950276,60.50281419700656
AdaBoostClassifier,ADASYN,auto,145.78613072514108,68.32029124029737
LogisticRegression,ADASYN,auto,153.81145063441457,90.14528132390727
LinearDiscriminantAnalysis,ADASYN,auto,154.12691534526903,92.01563844877147
RidgeClassifier,ADASYN,auto,153.31122394979778,86.43318091540166
GaussianNB,ADASYN,auto,150.96207517200153,92.6791318706037
SGDClassifier,ADASYN,auto,148.08539275562035,77.71473153369479
QuadraticDiscriminantAnalysis,ADASYN,auto,100.0,0.0
DummyClassifier,ADASYN,auto,100.0,0.0

In [ ]:
cm_total

In [ ]:

predictions = predict_model(best, data=test_features)
cm = confusion_matrix(test_labels, predictions["prediction_label"])
cm_total = cm.sum(axis=1)
result = [[0.0,0.0,0.0],[0.0,0.0,0.0],[0.0,0.0,0.0]]    
for row in range(0,3):
    for column in range(0,3):
        result[row][column] = cm[row][column]/(cm_total[row])
        
sns.heatmap(result, annot=True, 
            fmt='.2%', cmap='Blues')

In [ ]:
predictions = predict_model(best, data=test_features)
predictions.head()

In [ ]:
plot_model(best, plot = 'confusion_matrix')

In [ ]:
plot_model(best, plot = 'boundary')

In [ ]:
plot_model(best, plot = 'class_report')

In [ ]:
plot_model(best, plot = 'feature')

In [ ]:
plot_model(best, plot = 'learning')

## Setup
This function initializes the training environment and creates the transformation pipeline. Setup function must be called before executing any other function in PyCaret. It only has two required parameters i.e. `data` and `target`. All the other parameters are optional.

In [ ]:
# import pycaret classification and init setup



Once the setup has been successfully executed it shows the information grid containing experiment level information. 

- **Session id:**  A pseudo-random number distributed as a seed in all functions for later reproducibility. If no `session_id` is passed, a random number is automatically generated that is distributed to all functions.<br/>
<br/>
- **Target type:**  Binary, Multiclass, or Regression. The Target type is automatically detected. <br/>
<br/>
- **Label Encoding:**  When the Target variable is of type string (i.e. 'Yes' or 'No') instead of 1 or 0, it automatically encodes the label into 1 and 0 and displays the mapping (0 : No, 1 : Yes) for reference. In this tutorial, no label encoding is required since the target variable is of numeric type. <br/>
<br/>
- **Original data shape:**  Shape of the original data prior to any transformations. <br/>
<br/>
- **Transformed train set shape :**  Shape of transformed train set <br/>
<br/>
- **Transformed test set shape :**  Shape of transformed test set <br/>
<br/>
- **Numeric features :**  The number of features considered as numerical. <br/>
<br/>
- **Categorical features :**  The number of features considered as categorical. <br/>

PyCaret has two set of API's that you can work with. (1) Functional (as seen above) and (2) Object Oriented API.

With Object Oriented API instead of executing functions directly you will import a class and execute methods of class.

You can use any of the two method i.e. Functional or OOP and even switch back and forth between two set of API's. The choice of method will not impact the results and has been tested for consistency.

## Compare Models

This function trains and evaluates the performance of all the estimators available in the model library using cross-validation. The output of this function is a scoring grid with average cross-validated scores. Metrics evaluated during CV can be accessed using the `get_metrics` function. Custom metrics can be added or removed using `add_metric` and `remove_metric` function.

In [ ]:
# compare baseline models
best = compare_models()

## Analyze Model

You can use the `plot_model` function to analyzes the performance of a trained model on the test set. It may require re-training the model in certain cases.

In [ ]:
sampled_data.diabetes.value_counts()

In [ ]:
# plot confusion matrix
plot_model(best, plot = 'confusion_matrix')

In [ ]:
# plot AUC
plot_model(best, plot = 'auc')

In [ ]:
# plot feature importance
plot_model(best, plot = 'feature')

In [ ]:
# check docstring to see available plots 
help(plot_model)

An alternate to `plot_model` function is `evaluate_model`. It can only be used in Notebook since it uses ipywidget.

In [ ]:
evaluate_model(best)

## Prediction
The `predict_model` function returns `prediction_label` and `prediction_score` (probability of the predicted class) as new columns in dataframe. When data is `None` (default), it uses the test set (created during the setup function) for scoring.

In [ ]:
# predict on test set
holdout_pred = predict_model(best)

In [ ]:
# show predictions df
holdout_pred.head()

The same function works for predicting the labels on unseen dataset. Let's create a copy of original data and drop the `Class variable`. We can then use the new data frame without labels for scoring.

In [ ]:
# copy data and drop Class variable

new_data = data.copy()
new_data.drop('diabetes', axis=1, inplace=True)
new_data.head()

In [ ]:
# predict model on new_data
predictions = predict_model(best, data = new_data)
predictions.head()

## Save Model

Finally, you can save the entire pipeline on disk for later use, using pycaret's `save_model` function.

In [ ]:
# save pipeline
save_model(best, 'my_first_pipeline')

In [ ]:
# load pipeline
loaded_best_pipeline = load_model('my_first_pipeline')
loaded_best_pipeline

In [ ]:
# check all the available models
models()

In [ ]:
# train logistic regression with default fold=10
lr = create_model('lr')

The function above has return trained model object as an output. The scoring grid is only displayed and not returned. If you need access to the scoring grid you can use `pull` function to access the dataframe.

In [ ]:
lr_results = pull()
print(type(lr_results))
lr_results

In [ ]:
# train logistic regression with fold=3
lr = create_model('lr', fold=3)

In [ ]:
# train logistic regression with specific model parameters
create_model('lr', C = 0.5, l1_ratio = 0.15)

In [ ]:
# train lr and return train score as well alongwith CV
create_model('lr', return_train_score=True)

Some other parameters that you might find very useful in `create_model` are:

- cross_validation
- engine
- fit_kwargs
- groups

You can check the docstring of the function for more info.

In [ ]:
# help(create_model)

## ✅ Tune Model

This function tunes the hyperparameters of the model. The output of this function is a scoring grid with cross-validated scores by fold. The best model is selected based on the metric defined in optimize parameter. Metrics evaluated during cross-validation can be accessed using the `get_metrics` function. Custom metrics can be added or removed using `add_metric` and `remove_metric` function.

In [ ]:
# train a dt model with default params
dt = create_model('dt')

In [ ]:
# tune hyperparameters of dt
tuned_dt = tune_model(dt)

Metric to optimize can be defined in `optimize` parameter (default = 'Accuracy'). Also, a custom tuned grid can be passed with `custom_grid` parameter. 

In [ ]:
dt

In [ ]:
# define tuning grid
dt_grid = {'max_depth' : [None, 2, 4, 6, 8, 10, 12]}

# tune model with custom grid and metric = F1
tuned_dt = tune_model(dt, custom_grid = dt_grid, optimize = 'F1')

In [ ]:
# to access the tuner object you can set return_tuner = True
tuned_dt, tuner = tune_model(dt, return_tuner=True)

In [ ]:
# model object
tuned_dt

In [ ]:
# tuner object
tuner

The default search algorithm is `RandomizedSearchCV` from `sklearn`. This can be changed by using `search_library` and `search_algorithm` parameter.

In [ ]:
# tune dt using optuna
tuned_dt = tune_model(dt, search_library = 'optuna')

For more details on all available `search_library` and `search_algorithm` please check the docstring. Some other parameters that you might find very useful in `tune_model` are:

- choose_better
- n_iter
- early_stopping
- groups

You can check the docstring of the function for more info.

In [ ]:
# help(tune_model)

## ✅ Ensemble Model

This function ensembles a given estimator. The output of this function is a scoring grid with CV scores by fold. Metrics evaluated during CV can be accessed using the `get_metrics` function. Custom metrics can be added or removed using `add_metric` and `remove_metric` function.

In [ ]:
# ensemble with bagging
ensemble_model(dt, method = 'Bagging')

In [ ]:
# ensemble with boosting
ensemble_model(dt, method = 'Boosting')

Some other parameters that you might find very useful in `ensemble_model` are:

- choose_better
- n_estimators
- groups
- fit_kwargs
- probability_threshold
- return_train_score

You can check the docstring of the function for more info.

In [ ]:
# help(ensemble_model)

## ✅ Blend Models

In [ ]:
best_recall_models_top3 = compare_models(sort = 'Recall', n_select = 3)

In [ ]:
# blend top 3 models
blend_models(best_recall_models_top3)

Some other parameters that you might find very useful in `blend_models` are:

- choose_better
- method
- weights
- fit_kwargs
- probability_threshold
- return_train_score

You can check the docstring of the function for more info.

In [ ]:
# help(blend_models)

## ✅ Stack Models

This function trains a meta-model over select estimators passed in the estimator_list parameter. The output of this function is a scoring grid with CV scores by fold. Metrics evaluated during CV can be accessed using the `get_metrics` function. Custom metrics can be added or removed using `add_metric` and `remove_metric` function.

In [ ]:
# stack models
stack_models(best_recall_models_top3)

Some other parameters that you might find very useful in `stack_models` are:

- choose_better
- meta_model
- method
- restack
- probability_threshold
- return_train_score

You can check the docstring of the function for more info.

In [ ]:
# help(stack_models)

## ✅ Plot Model

This function analyzes the performance of a trained model on the hold-out set. It may require re-training the model in certain cases.

In [ ]:
# plot class report
plot_model(best, plot = 'class_report')

In [ ]:
# to control the scale of plot
plot_model(best, plot = 'class_report', scale = 2)

In [ ]:
# to save the plot
plot_model(best, plot = 'class_report', save=True)

Some other parameters that you might find very useful in `plot_model` are:

- fit_kwargs
- plot_kwargs
- groups
- display_format

You can check the docstring of the function for more info.

In [ ]:
# help(plot_model)

## ✅ Interpret Model

This function analyzes the predictions generated from a trained model. Most plots in this function are implemented based on the SHAP (Shapley Additive exPlanations). For more info on this, please see https://shap.readthedocs.io/en/latest/

In [ ]:
# train lightgbm model
lightgbm = create_model('lightgbm')

In [ ]:
# interpret summary model
interpret_model(lightgbm, plot = 'summary')

In [ ]:
# reason plot for test set observation 1
interpret_model(lightgbm, plot = 'reason', observation = 1)

Some other parameters that you might find very useful in `interpret_model` are:

- plot
- feature
- use_train_data
- X_new_sample
- y_new_sample
- save

You can check the docstring of the function for more info.

In [ ]:
# help(interpret_model)

## ✅ Get Leaderboard

This function returns the leaderboard of all models trained in the current setup.

In [ ]:
# get leaderboard
lb = get_leaderboard()
lb

In [ ]:
# select the best model based on F1
lb.sort_values(by='F1', ascending=False)['Model'].iloc[0]

Some other parameters that you might find very useful in `get_leaderboard` are:

- finalize_models
- fit_kwargs
- model_only
- groups

You can check the docstring of the function for more info.

In [ ]:
# help(get_leaderboard)

## ✅ AutoML
This function returns the best model out of all trained models in the current setup based on the optimize parameter. Metrics evaluated can be accessed using the `get_metrics` function.

In [ ]:
automl()

## ✅ Dashboard
The dashboard function generates the interactive dashboard for a trained model. The dashboard is implemented using `ExplainerDashboard`. For more information check out [Explainer Dashboard.](explainerdashboard.readthedocs.io)

In [ ]:
# dashboard function
dashboard(dt, display_format ='inline')

## ✅Create App
This function creates a basic gradio app for inference.

In [ ]:
# create gradio app
create_app(best)

## ✅ Create API
This function takes an input model and creates a POST API for inference.

In [ ]:
# create api
create_api(best, api_name = 'my_first_api')

In [ ]:
# !python my_first_api.py

In [ ]:
# check out the .py file created with this magic command
# %load my_first_api.py

## ✅ Create Docker
This function creates a `Dockerfile` and `requirements.txt` for productionalizing API end-point.

In [ ]:
create_docker('my_first_api')

In [ ]:
# check out the DockerFile file created with this magic command
# %load DockerFile

In [ ]:
# check out the requirements file created with this magic command
# %load requirements.txt

## ✅ Finalize Model
This function trains a given model on the entire dataset including the hold-out set.

In [ ]:
final_best = finalize_model(best)

In [ ]:
final_best

## ✅ Convert Model
This function transpiles the trained machine learning model's decision function in different programming languages such as Python, C, Java, Go, C#, etc. It is very useful if you want to deploy models into environments where you can't install your normal Python stack to support model inference.

In [ ]:
# transpiles learned function to java
print(convert_model(dt, language = 'java'))

## ✅ Deploy Model
This function deploys the entire ML pipeline on the cloud.

**AWS:**  When deploying model on AWS S3, environment variables must be configured using the command-line interface. To configure AWS environment variables, type `aws configure` in terminal. The following information is required which can be generated using the Identity and Access Management (IAM) portal of your amazon console account:

- AWS Access Key ID
- AWS Secret Key Access
- Default Region Name (can be seen under Global settings on your AWS console)
- Default output format (must be left blank)

**GCP:** To deploy a model on Google Cloud Platform ('gcp'), the project must be created using the command-line or GCP console. Once the project is created, you must create a service account and download the service account key as a JSON file to set environment variables in your local environment. Learn more about it: https://cloud.google.com/docs/authentication/production

**Azure:** To deploy a model on Microsoft Azure ('azure'), environment variables for the connection string must be set in your local environment. Go to settings of storage account on Azure portal to access the connection string required.
AZURE_STORAGE_CONNECTION_STRING (required as environment variable)
Learn more about it: https://docs.microsoft.com/en-us/azure/storage/blobs/storage-quickstart-blobs-python?toc=%2Fpython%2Fazure%2FTOC.json

In [ ]:
# deploy model on aws s3
# deploy_model(best, model_name = 'my_first_platform_on_aws',
#             platform = 'aws', authentication = {'bucket' : 'pycaret-test'})

In [ ]:
# load model from aws s3
# loaded_from_aws = load_model(model_name = 'my_first_platform_on_aws', platform = 'aws',
#                              authentication = {'bucket' : 'pycaret-test'})

# loaded_from_aws

## ✅ Save / Load Model
This function saves the transformation pipeline and a trained model object into the current working directory as a pickle file for later use.

In [ ]:
# save model
save_model(best, 'my_first_model')

In [ ]:
# load model
loaded_from_disk = load_model('my_first_model')
loaded_from_disk

## ✅ Save / Load Experiment
This function saves all the experiment variables on disk, allowing to later resume without rerunning the setup function.

In [ ]:
# save experiment
save_experiment('my_experiment')

In [ ]:
# load experiment from disk
exp_from_disk = load_experiment('my_experiment', data=data)